In [1]:
# Импорты библиотек
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')  # Чтобы не мешали предупреждения

In [ ]:
# Определение функции classification_training
def classification_training(data):
    # Преобразование целевой переменной
    y = (data['mental_wellness_index_0_100'] >= 15).astype(int)

    # Подготовка признаков. Исключаем целевую переменную и идентификатор пользователя
    feature_columns = [col for col in data.columns if col not in ['mental_wellness_index_0_100', 'user_id']]
    X = data[feature_columns].copy()

In [ ]:
# Обработка категориальных признаков. Преобразуем строковые значения в числа.
    categorical_features = ['gender', 'occupation', 'work_mode']
    for feature in categorical_features:
        if feature in X.columns:
            le = LabelEncoder()
            X[feature] = le.fit_transform(X[feature].astype(str))

In [ ]:
 # Разделение данных на обучающую и тестовую выборки
    # stratify=y — сохраняет пропорции классов в обеих выборках
    X_train, X_test, y_train, y_test = train_test_split(
        X, y,
        test_size=0.25,
        random_state=42,
        stratify=y
    )

In [ ]:
# Определение сетки гиперпараметров для поиска
    # n_estimators: количество деревьев в лесу (не слишком много, чтобы не переобучать)
    # max_depth: максимальная глубина дерева (ограничиваем переобучение)
    # min_samples_split: минимальное количество образцов для разбиения узла
    # min_samples_leaf: минимальное количество образцов в листе
    # max_features: количество признаков, рассматриваемых при поиске лучшего разбиения
    param_grid = {
        'n_estimators': [50, 100, 150], # Не ставим слишком много деревьев
        'max_depth': [5, 8, 12, None], # Ограничиваем глубину, None - без ограничений
        'min_samples_split': [5, 10, 15], # Минимум 5-15 образцов для разбиения
        'min_samples_leaf': [2, 4, 6], # Минимум 2-6 образцов в листе
        'max_features': ['sqrt', 'log2'] # Количество признаков для каждого разбиения
    }

In [ ]:
# Создание экземпляра случайного леса
# Random Forest — это ансамблевый метод, который объединяет несколько деревьев
    rf = RandomForestClassifier(random_state=42)

In [ ]:
# Подбор гиперпараметров с помощью GridSearchCV
    # GridSearchCV перебирает все комбинации параметров и выбирает лучшие по F1-мере
    # cv=3 — трехкратная кросс-валидация для оценки качества
    # n_jobs=-1 — используем все доступные ядра процессора для ускорения
    grid_search = GridSearchCV(
        estimator=rf,
        param_grid=param_grid,
        cv=3,
        scoring='f1', # Оптимизируем по F1, так как задача бинарной классификации
        n_jobs=-1,
        verbose=0  # Не выводим прогресс
    )
    grid_search.fit(X_train, y_train)

In [ ]:
# Вывод оптимального набора гиперпараметров
    # Это лучшая комбинация параметров, найденная GridSearchCV
    print("Best parameters found:")
    for param, value in grid_search.best_params_.items():
        print(f"  {param}: {value}")

In [ ]:
# Оценка финальной модели на тестовом наборе
    # Используем модель с лучшими параметрами для предсказания
    best_model = grid_search.best_estimator_
    y_pred = best_model.predict(X_test)
    rf_accuracy = accuracy_score(y_test, y_pred)
    rf_f1 = f1_score(y_test, y_pred)

In [ ]:
# Вывод метрик в заданном формате
    print(f"RF: {rf_accuracy:.4f}; {rf_f1:.4f}")

In [2]:
#data = pd.read_csv('processed_DB_3.csv')
#classification_training(data)

Best parameters found:
  max_depth: 8
  max_features: sqrt
  min_samples_leaf: 2
  min_samples_split: 10
  n_estimators: 100
RF: 0.8791; 0.8764
